In [ ]:
%matplotlib inline
%time from hikyuu.interactive import *
#use_draw_engine('echarts') #use_draw_engine('matplotlib')  #默认为'matplotlib'绘图

In [ ]:
import lxy.livemore as lm
import imp

imp.reload(lm)
for stock in sm:
    # Your code here
    kdata = stock.get_kdata(Query()).to_df()
    myLm = lm.livemore(kdata)
    myLm.livemoreProcess()
    
    counter = 0
    if len(myLm.df) < 250:
        continue
    for index, row in myLm.df.iterrows():
        
        time = index
        price = row['price']
        status = row['current_state']
        # Your code here
        # print(f"Price: {price}, Status: {status}")
        if status == 0:
            counter += 1
        else:
            counter = 0

    if counter <= 3 and counter > 0:
        print(stock.name, stock.code, counter)
        


In [ ]:
name = 'sz002578'
s = sm[name]
print(s.market)
# print(pd.Series(s.get_kdata(Query(-6)).close))
# print(pd.Series(s.get_kdata(Query(-6)).close).pct_change())
s = sm[name]
kdata = s.get_kdata(Query())
closing_prices = pd.Series(kdata.close)
datetime_array = [v.datetime for v in kdata]

# print("Closing Prices:")
# print(closing_prices)
# print("Time:")
# print(datetime_array)
# 代码逻辑：
# 1.正处于趋势区（上升/下降 趋势）。那么6点反转
# 2.正处于自然区
#     2.1 自然区突破：自然区如果超过同向的趋势区划线，或者超过自然区划线的3点，那么重新进入趋势区
#     2.2 自然区反转：自然区反向6点进入反转。自然区如果反转，那么进入次级区
# 3.正处于次级区（防止进入自然区后再反转，影响自然区划线）
#     3.1 次级去如果超过同向划线，进入同向自然区
#     3.2 次级区如果超过反向划线，进入反向自然区

# Define the state machine with six states
state_machine = {
    0: '上升趋势',
    1: '下降趋势',
    2: '自然上升',
    3: '自然下降',
    4: '次级上升',
    5: '次级下降'
}

# Set the initial state to '下降趋势'
current_state = 1

# Set the variables for the maximum values of each state
max_up = 0  # 上升趋势最大值
min_down = 0  # 下降趋势最小值
max_natrl_up = 0  # 自然上升最大值
min_natrl_down = 0  # 自然下降最小值

up_line = 0
down_line = 0  # 下降趋势最小值
natrl_up_line = 0  # 自然上升最大值
natrl_down_line = 0  # 自然下降最小值

# Iterate over the closing prices
# for price in closing_prices:
for i, price in enumerate(closing_prices):
    time = datetime_array[i]
    # Perform state transition based on the current state and closing price

    print('price:' + str(round(price, 2)) + '\t time:' + str(time) + '  stat:' + state_machine[current_state])
    if current_state == 0:
        # 上升趋势
        # Perform actions for 上升趋势 state
        if max_up == 0 or price > max_up:
            max_up = price
        # 趋势反转6点认为趋势结束，进入自然区间
        if price < max_up * (1 - 0.066):
            print("上升趋势->自然下降|六点反转|[最高点:" + str(round(max_up,2)), "][最高点六分位:", str(round(max_up * (1 - 0.066), 2)), " 当前价格:", str(round(price, 2)), "]" )
            up_line = max_up
            current_state = 3
            max_up = 0
        pass
    elif current_state == 1:
        # 下降趋势
        # Perform actions for 下降趋势 state
        if min_down == 0 or price < min_down:
            min_down = price

        # 趋势反转6点认为趋势结束，进入自然区间
        if price > min_down * (1 + 0.066):
            print("下降趋势->自然上升|六点反转|[最低点:" + str(round(min_down, 2)), "][最低六分位:", str(round(min_down * (1 + 0.066), 2)), "][当前价格:", str(round(price,2)), "]")
            down_line = min_down
            current_state = 2
            min_down = 0
        pass
    elif current_state == 2:
        # 自然上升
        # Perform actions for 自然上升 state
        if max_natrl_up == 0 or price > max_natrl_up:
            max_natrl_up = price

        # 自然区价格超过趋势最值，认为重新进入趋势
        if (up_line != 0 and price > up_line) or \
            (natrl_up_line != 0 and price > natrl_up_line * (1 + 0.033)):
            print("自然上升->上升趋势|三点突破|[趋势高点:" + str(round(up_line, 2)), "][自然高点:" + str(round(natrl_up_line, 2)), "][自然高点三分位:", str(round(natrl_up_line * (1 + 0.033))), "][当前价格:", str(round(price,2)), "]")
            current_state = 0
            natrl_up_line = price
            max_natrl_up = 0
            continue

        # 自然区反转，进入次级
        if price < max_natrl_up * (1 - 0.066):
            print("自然上升->次级下降|六点转向|[最高点:" + str(round(max_natrl_up, 2)), "][最高点六分位:", str(round(max_natrl_up * (1 - 0.066), 2)), "][当前价格:", str(round(price, 2)), "]")
            current_state = 5
            natrl_up_line = price
            max_natrl_up = 0

        pass
    elif current_state == 3:
        # 自然下降
        # Perform actions for 自然下降 state
        if min_natrl_down == 0 or price < min_natrl_down:
            min_natrl_down = price

        # 自然区价格超过趋势最值，认为重新进入趋势
        if (down_line != 0 and price < down_line) or \
            (natrl_down_line != 0 and price < natrl_down_line * (1 - 0.033)):
            print("自然下降->下降趋势|三点突破|[趋势低点:" + str(round(down_line, 2)), "][自然低点:" + str(round(natrl_down_line, 2)), "][自然低点三分位:", str(round(natrl_down_line * (1 - 0.033))), "][当前价格:", str(round(price,2)), "]")
            current_state = 1
            natrl_down_line = price
            min_natrl_down = 0
            continue

        # 自然区反转，进入次级
        if  price > min_natrl_down * (1 + 0.066):
            print("自然下降->次级上升|六点转向|[最低点:" + str(round(min_natrl_down, 2)), "][最低点六分位:", str(round(min_natrl_down * (1 + 0.066), 2)), "][当前价格:", str(round(price, 2)), "]")
            current_state = 4
            natrl_down_line = price
            min_natrl_down = 0

        pass

    elif current_state == 4:
        # 次级上升

        # 回归自然下降
        if price < natrl_down_line:
            print("次级上升->自然下降||[自然下降低点:" + str(round(natrl_down_line, 2)),"][当前价格:", str(round(price, 2)), "]")
            current_state = 3
            continue
        # 回归自然上升
        if natrl_up_line == 0 or price > natrl_up_line:
            print("次级上升->自然上升||[自然上升高点:" + str(round(natrl_up_line, 2)),"][当前价格:", str(round(price, 2)), "]")
            current_state = 2
            continue
        pass
    elif current_state == 5:
        # 次级下降

        # 回归自然上升
        if price > natrl_up_line:
            print("次级下降->自然上升||[自然上升高点:" + str(round(natrl_up_line, 2)),"][当前价格:", str(round(price, 2)), "]")
            current_state = 2
            continue
        # 回归自然下降
        if natrl_down_line == 0 or price < natrl_down_line:
            print("次级下降->自然下降||[自然下降低点:" + str(round(natrl_down_line, 2)),"][当前价格:", str(round(price, 2)), "]")
            current_state = 3
            continue
        pass


In [ ]:
data_map = {}
s = sm['bj873570']
print(s.market)
print(pd.Series(s.get_kdata(Query(-6)).close))
print(pd.Series(s.get_kdata(Query(-6)).close).pct_change())
for i, stock in enumerate(sm):
    # 不是A股的干掉
    if stock.type != constant.STOCKTYPE_A:
        continue

    if stock.market != 'SH' and stock.market != 'SZ':
        continue    
    kdata = stock.get_kdata(Query(-6))
    closing_prices = pd.Series(kdata.close)  # Convert to pandas Series
    datetime_array = []
    for v in k:
        datetime_array.append(v.datetime)

    # Calculate the daily returns
    closing_prices_length = len(closing_prices)
    if closing_prices_length == 0:
        print(stock.name)
        continue
    
    returns = closing_prices.pct_change()
    returns = returns[1:]

    # Store the values in the map
    for date, value in zip(datetime_array[1:], returns):
        if date not in data_map:
            data_map[date] = {}
        data_map[date][stock.name] = round(value, 5)

filtered_stocks = {}
for date, stocks in data_map.items():
    filtered_stocks[date] = {stock: value for stock, value in stocks.items() if value > 0.095}

for date, stocks in filtered_stocks.items():
    print(f"Date: {date}")
    print(f"Number of Stocks: {len(stocks)}")
    for stock, value in stocks.items():
        print(f"Stock: {stock}, Return: {value}")